In [19]:
import pandas as pd
import pyodbc 
from datetime import date,datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd
print('Librerias Cargadas.')
server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'mencarna' 
password = 'Pelus@141516'  
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
print('Conección Exitosa.')
Anio=2025
Mes=12
Tipo = "Avance"
if Tipo=="Avance":
    fechas = datetime(Anio,Mes, datetime.now().day)- timedelta (days = 1)
else:
    fechas = datetime(Anio,Mes,1)+ relativedelta(months=+1)- timedelta (days = 1)
print('Variables del proceso: Anio: '+str(Anio)+', Mes: '+ str(Mes)+'.')

Librerias Cargadas.
Conección Exitosa.
Variables del proceso: Anio: 2025, Mes: 12.


In [20]:
query = f"""
/*********************************************AFILIACIONES APP*******************************************/

select A.Usuario,A.Anio,A.Mes,iif(B.Real is null,0,B.Real) Real,'AFILIACIONES APP' as Producto
from
(select ltrim(rtrim(upper(usuarioOportunidad))) Usuario,count(Identificacion) Real,
'APP MOVIL' as Producto
from [DATALAB_IDN].[IDN].[MasivoBancaMovil]
where motivo='Efectivo' and year(fechaCierre)={Anio} and month(fechacierre)={Mes}
group by ltrim(rtrim(upper(usuarioOportunidad)))
) AS B 
right JOIN (
select ANIO,MES,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,Agencia,Usuario,Asesor,
Medibles,PorcMedicion,ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion 
from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' 
and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')
) AS A 
ON A.USUARIO=B.USUARIO

UNION ALL
/*********************************************SEGUROS*******************************************/
SELECT A.Usuario,A.Anio,A.Mes,iif(B.Real is null,0,B.Real) Real,'SEGUROS' as Producto FROM 
(
SELECT aliasPropietarioOportunidad AliasDelPropietarioDeLaOpor,
sum(ValorPlan) as Real FROM [DATALAB_IDN].[IDN].[MasivoSegurosNovaStratio]
where 
TIPOOPERACION='CONSIDERADO' 
and year(fechaCierre)={Anio} and month(fechaCierre)={Mes}
group by aliasPropietarioOportunidad
) 
AS B right JOIN ( select ANIO,MES,REGION,CodigoAgEjecucion,
CodigoAgencia,Zona,AgenciaEjecucion,Agencia,Usuario,Asesor,Medibles,PorcMedicion,ProvinciaAgencia,CiudadAgencia,
CategorizacionAgencia,TipoMedicion from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and 
TipoAsesorProductividad='MULTIFUNCION' and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')) 

AS A ON A.USUARIO=B.AliasDelPropietarioDeLaOpor
UNION ALL
/*****************************************GESTION COBRANZA LLAMADAS***********************************/
SELECT A.Usuario,A.Anio,A.Mes,iif(B.Real is null,0,B.Real) Real,'GESTION COBRANZA LLAMADAS' as Producto 
from
(select * FROM  [DATALAB_IDN].[HIS].GESTION_COBRANZAS_LLAMADAS AS A
where  Anio={Anio} and Mes={Mes}) B right JOIN (select ANIO,MES,REGION,CodigoAgEjecucion,
CodigoAgencia,Zona,AgenciaEjecucion,Agencia,Usuario,Asesor,Medibles,PorcMedicion,ProvinciaAgencia,CiudadAgencia,
CategorizacionAgencia,TipoMedicion from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' 
and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')) AS A 
ON A.USUARIO=B.USUARIO

UNION ALL

/*********************************************REMESAS*******************************************/
SELECT 
DT01.Usuario, 
Dt01.Anio,
Dt01.Mes,
SUM(A.Valor) AS Real, 
'REMESAS' AS Producto
FROM
( 
SELECT  A.FechaAnalisis,A.CodigoAgencia,A.Usuario,A.Valor,A.VALIDADOR 
 FROM [DATALAB_IDN].[IDN].[SEGUIMIENTO_REMESAS] AS A WITH (NOLOCK)
 UNION ALL
 SELECT A.FechaAnalisis,A.CodigoAgencia,A.Usuario,A.Valor,A.VALIDADOR 
 FROM [DATALAB_IDN].[IDN].[SEGUIMIENTO_REMESAS_NOCLIENTES] AS A WITH (NOLOCK)
 UNION ALL
 SELECT A.FechaAnalisis, A.CodigoAgencia,A.Usuario, A.Valor, A.VALIDADOR
		FROM [DATALAB_IDN].[IDN].[SEGUIMIENTO_REMESAS_DIGITAL] AS A WITH (NOLOCK)
) AS A
LEFT JOIN 
(
SELECT C.Usuario, C.CodigoAgEjecucion,
C.TipoAsesorProductividad ,C.TipoMedicion,
C.Mes,C.Anio, C.Observaciones
FROM [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias] AS C WITH (NOLOCK)
WHERE Anio = {Anio} AND Mes = {Mes}
) AS DT01
ON DT01.Usuario = A.Usuario
WHERE A.VALIDADOR = 'EFECTIVO'
AND YEAR(A.FechaAnalisis) = {Anio} AND MONTH(A.FechaAnalisis) = {Mes}
AND DT01.TipoAsesorProductividad = 'MULTIFUNCION' and 
Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')
GROUP BY 
DT01.Usuario,
Dt01.Anio,
Dt01.Mes

UNION ALL
/*****************************************DESEMBOLSOS******************************************/
select 
A.Usuario,A.Anio,A.Mes,iif(B.Real is null,0,B.Real) Real,'DESEMBOLSOS' AS Producto
from 
(
SELECT ltrim(rtrim(upper(aliasOportunidad))) AS Usuario,
cast(round(sum(montoOriginal),0) as numeric(23,2)) as Real
FROM [DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadMasivoCartera as A
LEFT JOIN [DATALAB_IDN].[IDN].[LISTA_OBSERVADOS] AS b
ON a.identificacion=b.identificacion
where
b.identificacion is null and
TIPOOPERACION='CONSIDERADO' and year(fechaCierre)={Anio} and month(fechaCierre)={Mes}
group by ltrim(rtrim(upper(aliasOportunidad)))
) AS B 
right JOIN 
(select ANIO,MES,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,
Agencia,Usuario,Asesor,Medibles,PorcMedicion,
ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion 
from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')) AS A 
ON A.Usuario=B.Usuario

UNION ALL
/*****************************************EFECTIVO EXPRESS******************************************/
select 
A.Usuario,
A.Anio,
A.Mes,
iif(B.Real is null,0,B.Real) Real,
'EFECTIVO EXPRESS' as Producto
from
(
select ANIO,MES,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,
Agencia,Usuario,Asesor,Medibles,PorcMedicion,
ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion 
from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')
) AS A 
left join
(
select
	Usuario,
	sum(REAL) as REAL--,
from [DATALAB_IDN].[IDN].[EFECTIVOXPRESSV2] A with (nolock) 
where month(fechaProceso) = {Mes} and year(fechaProceso) = {Anio} AND observacionValidacion='CONSIDERADO'
--and tipoAsesorProductividad in ('PREFERENCIAL', 'OFICIAL COMERCIAL Y SERVICIOS VIRTUAL')
group by USUARIO
) b on a.Usuario=b.USUARIO

UNION ALL
/*****************************************AHORRO TOTAL******************************************/
select 
A.Usuario,A.Anio,A.Mes,iif(B.Real is null,0,B.Real) Real,'AHORRO TOTAL' AS Producto
from 
(
SELECT ltrim(rtrim(upper(aliasOportunidad))) AS Usuario,
cast(round(sum(incremento),0) as numeric(23,2)) as Real
FROM 
(
SELECT * FROM (
SELECT 
aliasOportunidad,a.identificacion,incremento,
ROW_NUMBER() OVER(PARTITION BY aliasOportunidad,a.identificacion ORDER BY 
DATEFROMPARTS(year(fechaCierre),MONTH(fechaCierre),1) DESC) rownumber
FROM [DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadAhorroFlexible AS a
LEFT JOIN [DATALAB_IDN].[IDN].[LISTA_OBSERVADOS] AS b
ON a.identificacion=b.identificacion
where
b.identificacion is null and
TipoOperacion='CONSIDERADO' AND --consideracion='SI' and 
year(fechaCierre)={Anio} and month(fechaCierre)={Mes}
) AS B
where rownumber=1
) AS a
group by ltrim(rtrim(upper(aliasOportunidad)))
) AS B 
right JOIN 
(select ANIO,MES,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,
Agencia,Usuario,Asesor,Medibles,PorcMedicion,
ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion 
from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')) AS A 
ON A.Usuario=B.Usuario
"""
df_Real = pd.read_sql(query, cnxn)

print('La obtención de Datos Reales de la Fuerza de Venta es Exitosa.')
query = f"""
select Anio,Mes,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,Agencia,Usuario,Asesor,Medibles,PorcMedicion,
ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' 
and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')

"""
df_HeatCount = pd.read_sql(query, cnxn)
print('La obtención del Heat Count de la Fuerza de Venta es Exitosa.')

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\3400010928.py:175: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Real = pd.read_sql(query, cnxn)


La obtención de Datos Reales de la Fuerza de Venta es Exitosa.
La obtención del Heat Count de la Fuerza de Venta es Exitosa.


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\3400010928.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_HeatCount = pd.read_sql(query, cnxn)


In [21]:
server = 'ECBPPRQ121,11124' 
database = 'BDDELIVERY' 
username = 'erisimba' 
password = 'Nomasclaves2024#' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
print('Conección Exitosa.')
query = f"""
/*****************************************AHORRO PROGRAMADO******************************************/
SELECT ltrim(rtrim(upper(aliasOportunidad))) AS Usuario,
cast(round(sum(incremento),0) as numeric(23,2)) as Real,
'AHORRO PROGRAMADO' AS Producto
FROM 
(
SELECT * FROM (
SELECT 
aliasOportunidad,a.identificacion,incremento,
ROW_NUMBER() OVER(PARTITION BY aliasOportunidad,a.identificacion ORDER BY 
DATEFROMPARTS(year(fechaCierre),MONTH(fechaCierre),1) DESC) rownumber
FROM 
DATALAB_IDN.[IDN].[MasivoAhorro1] AS A
--BDDELIVERY.IDN.MasivoAhorro AS a
LEFT JOIN [DATALAB_IDN].[IDN].[LISTA_OBSERVADOS] AS b
ON a.identificacion=b.identificacion
where
b.identificacion is null and
consideracion='SI' and TipoOperacion='CONSIDERADO' AND
year(fechaCierre)={Anio} and month(fechaCierre)={Mes}
) AS B
where rownumber=1
) AS a
group by ltrim(rtrim(upper(aliasOportunidad)))

union all


/*****************************************TARJETAS DE CREDITO******************************************/
SELECT ltrim(rtrim(upper(aliasOportunidad))) AS Usuario,
cast(COUNT(TARJETASOCIO) as numeric(23,2)) as Real,
'TARJETAS' AS Producto
FROM DATALAB_IDN.[IDN].[MasivoTarjeta1] as A
LEFT JOIN [DATALAB_IDN].[IDN].[LISTA_OBSERVADOS] AS b
ON a.identificacion=b.identificacion
where
b.identificacion is null and
TIPOOPERACION='CONSIDERADO' and year(fechaCierre)={Anio} and month(fechaCierre)={Mes}
group by ltrim(rtrim(upper(aliasOportunidad)))
-------------------- PLAZO ------------------
UNION ALL
SELECT DT.Usuario, DT01.Real, 'PLAZO' AS PRODUCTO FROM (
select ANIO,MES,REGION,CodigoAgEjecucion,
CodigoAgencia,Zona,AgenciaEjecucion,Agencia,Usuario,Asesor,Medibles,PorcMedicion,ProvinciaAgencia,CiudadAgencia,
CategorizacionAgencia,TipoMedicion from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' 
and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')
) DT
LEFT JOIN 
(
	select usuarioOportunidad AS Usuario, SUM(montoApertura) AS Real from 
    DATALAB_IDN.[IDN].[MasivoPlazo1]
WHERE YEAR(fechaCierre) = {Anio} AND MONTH(fechaCierre) = {Mes}
GROUP BY usuarioOportunidad
) DT01
ON DT.Usuario = DT01.Usuario
"""
df_Nube = pd.read_sql(query, cnxn)
print('La obtención de Información de la Nube ha sido Exitosa.')

Conección Exitosa.


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\2691694628.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Nube = pd.read_sql(query, cnxn)


La obtención de Información de la Nube ha sido Exitosa.


In [22]:
df1 = pd.merge(df_HeatCount, df_Nube[df_Nube.Producto=='AHORRO PROGRAMADO'],  how='left', 
             left_on=['Usuario'], 
             right_on =['Usuario'] )
df1['Producto']='AHORRO PROGRAMADO'
df1 = df1[['Usuario','Anio','Mes','Real','Producto']]

df4 = pd.merge(df_HeatCount, df_Nube[df_Nube.Producto=='TARJETAS'],  how='left', 
             left_on=['Usuario'], 
             right_on =['Usuario'] )
df4['Producto']='TARJETAS'
df4 = df4[['Usuario','Anio','Mes','Real','Producto']]

df5 = pd.merge(df_HeatCount, df_Nube[df_Nube.Producto=='PLAZO'],  how='left', 
             left_on=['Usuario'], 
             right_on =['Usuario'] )
df5['Producto']='PLAZO'
df5 = df5[['Usuario','Anio','Mes','Real','Producto']]

df_Final= pd.concat([df_Real, df1,df4, df5])

df_Final["Real"].fillna(0, inplace=True)

display(df_Final["Producto"].unique())

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\507262532.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_Final= pd.concat([df_Real, df1,df4, df5])
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\507262532.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_Fi

array(['AFILIACIONES APP', 'SEGUROS', 'GESTION COBRANZA LLAMADAS',
       'REMESAS', 'DESEMBOLSOS', 'EFECTIVO EXPRESS', 'AHORRO TOTAL',
       'AHORRO PROGRAMADO', 'TARJETAS', 'PLAZO'], dtype=object)

In [23]:
df = pd.read_excel('//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Masivo/Pool.xlsx',
                   header=[0,1,2],
                   index_col=[0],
                   sheet_name='Metas')
df = df.stack(level=1).stack(level=1).stack().unstack(level=1)
df = df.reset_index()
df=df.rename(columns = {'USUARIO':'PRODUCTO','level_0':'USUARIO','level_2':'PRODUCTOS'})
df = df[['PRODUCTO','USUARIO','METAS']]
df_BSC = pd.merge(df,df_Final,  how='left', 
             left_on=['PRODUCTO','USUARIO'], 
             right_on =['Producto','Usuario'] )
df_BSC =df_BSC[['PRODUCTO','USUARIO','Real','METAS']]
df = pd.read_excel('//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Masivo/Pool.xlsx',
                   sheet_name='Logica')
df_Final = pd.merge(df_BSC,df,  how='left', 
             left_on=['PRODUCTO'], 
             right_on =['PRODUCTO'] )
import pandas as pd
import datetime
df_Final['FECHA_CORTE'] = pd.to_datetime(fechas)
#pd.Timestamp(datetime.date.today() - timedelta(days=10))
df_Final = df_Final.drop(['ANIO', 'MES'], axis=1)

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\92407218.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).stack(level=1).stack().unstack(level=1)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\92407218.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).stack(level=1).stack().unstack(level=1)


In [24]:
df_Final["Real"].fillna(0, inplace=True)

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\3542581548.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_Final["Real"].fillna(0, inplace=True)


In [25]:
server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'erisimba' 
password = 'Nomasclaves2024#' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
print('Conección Exitosa.')
query = f"""
select REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,Agencia,Usuario,Asesor,Medibles,PorcMedicion,
ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion 
from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' 
and Observaciones in ('Oficial Comercial y Servicios Pool con rotación Metas Pool')
"""
df_HeatCount = pd.read_sql(query, cnxn)
df = pd.merge(df_Final,df_HeatCount,  how='left', 
             left_on=['USUARIO'], 
             right_on =['Usuario'] )
df = df.drop(['Usuario'], axis=1)
df = df[df['ACTIVO']=='SI']
df = df.fillna(0)

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\2310234148.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_HeatCount = pd.read_sql(query, cnxn)


Conección Exitosa.


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23324\2310234148.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


In [26]:
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Banca_Multifuncion_Pool] ")
for index, row in df.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Banca_Multifuncion_Pool] 
    (PRODUCTO,USUARIO,Real,METAS,CATEGORIA,ACTIVO,TECHO_PRODUCTO,TECHO_P,TECHO_C,PORCENTAJE,INVERTIBILIDAD,
    TIPO,VALOR,ERROR,FECHA_CORTE,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,Agencia,
    Asesor,Medibles,PorcMedicion,ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                   row.PRODUCTO,row.USUARIO,row.Real,row.METAS,row.CATEGORIA,row.ACTIVO,row.TECHO_PRODUCTO,
                   row.TECHO_P,row.TECHO_C,row.PORCENTAJE,row.INVERTIBILIDAD,row.TIPO,row.VALOR,row.ERROR,
                   row.FECHA_CORTE,row.REGION,row.CodigoAgEjecucion,row.CodigoAgencia,row.Zona,
                   row.AgenciaEjecucion,row.Agencia,row.Asesor,row.Medibles,row.PorcMedicion,
                   row.ProvinciaAgencia,row.CiudadAgencia,row.CategorizacionAgencia,row.TipoMedicion)
    cnxn.commit()
print('Completado')

Completado


In [27]:
import pandas as pd
import pyodbc 
from datetime import date,datetime, timedelta
from pandas.tseries.offsets import MonthEnd

server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'erisimba' 
password = 'Nomasclaves2024#' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
cursor.execute(f"""
DECLARE @MES INT,@ANIO INT
	DELETE FROM [DATALAB_IDN].[HIS].[Banca_Multifuncion_Pool] 
	WHERE YEAR(FECHA_CORTE) = {Anio} AND MONTH(FECHA_CORTE) = {Mes}
	INSERT INTO [DATALAB_IDN].[HIS].[Banca_Multifuncion_Pool] 
	SELECT * FROM  [DATALAB_IDN].[IDN].[Banca_Multifuncion_Pool] 
""")
cnxn.commit()
print('La Historia esta Cargada.')

La Historia esta Cargada.
